# Conditional GAN on Fashion-MNIST

This notebook demonstrates training a **Conditional Generative Adversarial Network (CGAN)** on the **Fashion-MNIST** dataset using TensorFlow 2.x. The model learns to generate grayscale images (28×28) of clothing items, conditioned on class labels.

---

## Setup & Imports


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython import display

from model import CGAN_MNIST_Fashion
import utils

## Load and preprocess Fashion-MNIST dataset


In [ ]:
(train_images, train_labels), (valid_images, valid_labels) = keras.datasets.fashion_mnist.load_data()

train_images = ((np.reshape(train_images, (-1, 28, 28, 1)).astype('float32')) / 255.0) * 2.0 - 1.0
valid_images = ((np.reshape(valid_images, (-1, 28, 28, 1)).astype('float32')) / 255.0) * 2.0 - 1.0

all_images = np.concatenate([train_images, valid_images], axis=0)
all_labels = np.concatenate([train_labels, valid_labels], axis=0)

BUFFER_SIZE = len(all_images)
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices((all_images, all_labels))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


## Model setup


In [ ]:
keras.backend.clear_session()
tf.random.set_seed(21)
np.random.seed(21)

LATENT_DIM = 100
LEARNING_RATE = 2e-3
CLASS_LABELS = 10
NUM_FEATURE_MAP = 256

model_cgan = CGAN_MNIST_Fashion(latent_dim=LATENT_DIM,
                                num_feature_map=NUM_FEATURE_MAP,
                                dropout=0.45,
                                alpha=0.2,
                                image_size=(28, 28, 1),
                                class_labels=CLASS_LABELS)

gen_optimizer = keras.optimizers.Adam(LEARNING_RATE, 0.5)
disc_optimizer = keras.optimizers.Adam(LEARNING_RATE, 0.5)

fixed_z = tf.random.normal([16, LATENT_DIM])
fixed_label = tf.random.uniform([16], minval=0, maxval=10, dtype=tf.int32)


## Train the CGAN model


In [ ]:
NUM_EPOCHS = 30

gen_losses, disc_losses, gen_grad_norms, disc_grad_norms = utils.train_model(
    model=model_cgan,
    gen_optimizer=gen_optimizer,
    disc_optimizer=disc_optimizer,
    train_dataset=train_dataset,
    num_epochs=NUM_EPOCHS,
    fixed_z=fixed_z,
    fixed_label=fixed_label,
    loss_type='vanilla',
    lambda_gp=10.0,
    clip_norm=True,
    max_norm=5.0
)


## Plot training curves


In [ ]:
df = pd.DataFrame({
    'gen_loss': gen_losses,
    'disc_loss': disc_losses,
    'gen_grad_norm': gen_grad_norms,
    'disc_grad_norm': disc_grad_norms
})

df[['gen_grad_norm', 'disc_grad_norm']].plot(figsize=(8, 5), title="Gradient Norms")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("L2 Norm")
plt.show()

df[['gen_loss', 'disc_loss']].plot(figsize=(8, 5), title="Generator and Discriminator Loss")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


## Generate sample images for selected classes


In [ ]:
for label in [1, 3, 7, 9]:
    utils.plot_generated_images_tf(model_cgan,
                                   digit_label=label,
                                   latent_dim=LATENT_DIM,
                                   grid_dim=4,
                                   dim1=0,
                                   dim2=1)


## Conclusion

In this notebook, we successfully trained a Conditional GAN (CGAN) on the Fashion-MNIST dataset.  
The model learned to generate realistic, class-conditional fashion images, demonstrated by the generated samples.  

We also monitored training stability through generator and discriminator loss curves as well as gradient norms.  
This CGAN framework can be further extended or adapted for other datasets and conditional image generation tasks.
